# 중요 결정사항 및 설명란
deltaprice = (t+1주가) - (t주가) / (t주가) --> 향후 1년 상승측정 
* 1. deltaprice에서 0은 상폐년도인 것 같음.. 확인 필요 왜냐면 주가 있다가 내년에 사라진거니깐 
* 2. deltaprice에서 99는 deltaprice연산때 Null값 나온거임 해당년도에 상장되어 있지 않기때문인 듯
* 3. Consensus 0(의견없음)이 존나 많음, 그리고 hold랑 sell이 전체년도에서 100개 수준인 것 같음
* 4. 처음에 Cut20NullfillNA에서 쓸모없는 데이터 버리는데, 여기서 기본 PEIS데이터만 있을때 20개 NULL값 있는거만 쓸모없었고 나머지는 0 채워서 쓰면댈 듯 근데 은행들 이런친구들은 아에안쓰는 항목들은 PEIS지표에 어떻게 영향을 줄진 모르겠음
* 5. 2010년 2021년 데이터는 없어도 댐 쓰면도 안댐

###     나중에 kospi, kosdaq 별로 따로 읽어서 처리 행야함 + 내 생각에 다중회귀분석은 따로 만들어야할 듯 

# 필요 모듈

In [1]:
import pandas as pd
import numpy as np
import warnings
from Fin import *
warnings.filterwarnings(action='ignore')

# 0. 엑셀읽기

In [2]:
df = pd.read_excel("Data/KospiPEISData&PriceData.xlsx")
df = Cut20NullfillNA(df)

# 여기서 deltaprice이상한거 제거하기
# 여기서부터 년도 별로 나누기 해야할 가능성 있음

In [3]:
df = df.drop(df[ (df['deltaprice']==99) | (df['deltaprice']==np.inf)].index)

# 1. PEIS를 구하기 위한 기초데이터들 연산해보기 (기업별로)

In [4]:
PEISBaseData = CompuMoreData(df)
PEISBaseData = setSGA(PEISBaseData)

# 2. 가공된 데이터 토대로 +1,-1,0 구하기 (년도별로)

In [6]:
Yearlydfs = yearbaseDFs(PEISBaseData, years =[2011,2012,2013,2014,2015,2016,2017,2018,2019,2020])
PEISBaseData = CalPEISpoints(Yearlydfs)

In [7]:
PEISBaseData

,index,Name,회계년,Symbol,총자산(천원),총자산(평균)(천원),영업이익(천원),매출원가(천원),판매비와관리비(천원),금융원가(비영업)(천원),...,deltaATO,SGAdeltasale>0,SGAdeltasale<0,GNOAPEISpoint,deltaATOPEISpoint,deltaGM2PEISpoint,deltaACCPEISpoint,RNOAPEISpoint,SGAdeltasale>0PEISpoint,SGAdeltasale<0PEISpoint
0,2,현대미포조선,2012,A010620,7.722829e+09,7.773696e+09,9.399499e+07,4.056670e+09,2.646859e+08,5.530287e+07,...,-0.019214,0.000000e+00,8.129713e+07,0,0,-1,-1,0,1,1
1,13,현대모비스,2012,A012330,3.004700e+10,2.631132e+10,2.906385e+09,2.624484e+10,1.637799e+09,1.475890e+08,...,-0.140037,4.225326e+09,0.000000e+00,-1,-1,0,0,-1,-1,1
2,24,LG카드,2012,A032710,2.227992e+10,2.231840e+10,9.561310e+08,0.000000e+00,6.234020e+08,0.000000e+00,...,-0.008221,0.000000e+00,-3.353000e+07,1,0,0,-1,-1,1,-1
3,35,현대건설,2012,A000720,1.274683e+10,1.230936e+10,7.604080e+08,1.207595e+10,4.884640e+08,1.412270e+08,...,0.041277,1.379842e+09,0.000000e+00,-1,0,0,0,0,-1,1
4,46,흥국화재,2012,A000540,5.875041e+09,5.244110e+09,6.043900e+07,1.032115e+09,5.357670e+08,0.000000e+00,...,-0.065876,9.400400e+07,0.000000e+00,1,0,0,1,-1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,8570,우진플라임,2020,A049800,2.676219e+08,2.677065e+08,9.434364e+06,1.518314e+08,4.373035e+07,4.817002e+06,...,0.087096,5.428244e+06,0.000000e+00,0,1,1,0,0,0,1
806,8581,동성케미컬,2020,A102260,7.979441e+08,8.242313e+08,6.117365e+07,7.189081e+08,1.019691e+08,9.890572e+06,...,0.106265,2.291678e+07,0.000000e+00,0,1,0,1,0,0,1
807,8592,일정실업,2020,A008500,4.676853e+07,5.364374e+07,-7.342536e+06,3.346066e+07,5.212205e+06,6.455610e+05,...,0.012417,0.000000e+00,-1.745034e+07,1,0,-1,1,1,1,0
808,8603,사조산업,2020,A007160,1.087399e+09,1.112405e+09,2.191712e+07,5.671177e+08,3.289265e+07,1.620804e+07,...,-0.074640,0.000000e+00,-1.171827e+08,0,0,0,0,0,1,-1


# 3. PEIS 점수 구하기 (단순 행 sum)

In [8]:
PEISBaseData = CalPEIS(PEISBaseData)

# 4. 년도 별 포트폴리오 구하기 위해 년도 별로 나누기

In [9]:
dfs = yearbaseDFs(PEISBaseData, years = [2011,2012,2013,2014,2015,2016,2017,2018,2019,2020])

## 5. 조건 & 년도별로 포트폴리오 수익률 구하기 (equally weight)

In [10]:
PEISbuyport = Upper20Port(dfs,"valPEIS")
np.mean(PEISbuyport)

0.12086963609933053

# 6. 엑셀로 보내기 

In [11]:
writer = pd.ExcelWriter('Result/Compute_PEIS.xlsx', engine='xlsxwriter')

In [12]:
collist=['ADJprice', 'Consensus',\
       'deltaprice', 'CountNull', 'NetAsset', 'deltaACC', 'RNOA', 'GNOA',\
       'deltaGNOA', 'SGA', 'deltaSGA', 'GM', 'deltaGM1', 'deltasale',\
       'deltaGM2', 'ATO', 'deltaATO', 'SGAdeltasale>0', 'SGAdeltasale<0',\
       'GNOAPEISpoint', 'deltaATOPEISpoint', 'deltaGM2PEISpoint',\
       'deltaACCPEISpoint', 'RNOAPEISpoint', 'SGAdeltasale>0PEISpoint',\
       'SGAdeltasale<0PEISpoint', 'valPEIS']

In [13]:
Computed = PEISBaseData[collist]

In [14]:
PEISBaseData.to_excel(writer,sheet_name = 'All_data')
Computed.to_excel(writer,sheet_name = 'Computed')

In [15]:
writer.save()

# 이 아래에서는 이것저것 실험